In [97]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Data/winequality-red-cleaned.csv', sep=',')

In [3]:
df.head()

,volatile acidity,residual sugar,chlorides,total sulfur dioxide,pH,sulphates,alcohol,quality,category
0,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246,5,Bad
1,1.967442,0.043416,0.223875,0.624363,-0.719933,0.128950,-0.584777,5,Bad
2,1.297065,-0.169427,0.096353,0.229047,-0.331177,-0.048089,-0.584777,5,Bad
3,-1.384443,-0.453218,-0.264960,0.411500,-0.979104,-0.461180,-0.584777,6,Good
4,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246,5,Bad


In [4]:
# Get labels and store in y variable
y = pd.DataFrame([0. if item == 'Good' else 1. for item in df['category']])
y = [0. if item == 'Good' else 1. for item in df['category']]
y[:5]

[1.0, 1.0, 1.0, 0.0, 1.0]

In [5]:
# Get features and store in X variable
X = df.drop(['quality', 'category'], axis=1)

In [6]:
X.head()

,volatile acidity,residual sugar,chlorides,total sulfur dioxide,pH,sulphates,alcohol
0,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246
1,1.967442,0.043416,0.223875,0.624363,-0.719933,0.128950,-0.584777
2,1.297065,-0.169427,0.096353,0.229047,-0.331177,-0.048089,-0.584777
3,-1.384443,-0.453218,-0.264960,0.411500,-0.979104,-0.461180,-0.584777
4,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246


In [7]:
import torch

In [8]:
# TODO: pip install sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train = features_df
# y_train = np.array(y)

/Users/kendall/Dropbox/Current_Projects/python/pytorch_tutorial/.direnv/python-3.5.2/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.activation import Softmax
import torch.optim as optim

In [51]:
learning_rate = 0.005

In [52]:
class SingleLayerNet(nn.Module):
    def __init__(self):
        super(SingleLayerNet, self).__init__()
        self.fc1 = nn.Linear(7, 2)

    def forward(self, x):
        x = F.softmax(self.fc1(x))
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net0 = SingleLayerNet()
optimizer0 = optim.SGD(net0.parameters(), lr=learning_rate, momentum=0.9)

In [53]:
class DoubleLayerNet(nn.Module):
    def __init__(self):
        super(DoubleLayerNet, self).__init__()
        self.fc1 = nn.Linear(7, 7)
        self.fc2 = nn.Linear(7, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = F.softmax(self.fc2(x))
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net1 = DoubleLayerNet()
optimizer1 = optim.SGD(net1.parameters(), lr=learning_rate, momentum=0.9)

In [54]:
batch_size = 100
epochs = 1000

In [55]:
criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss

In [56]:
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix, accuracy_score

In [61]:
def train_with_net(net, optimizer, criterion, batch_size, 
                   epochs):
    for epoch in range(epochs): # loop over the dataset multiple times
        
        running_loss = 0.0
        for start, end in zip(range(0, len(X_train), batch_size), 
                              range(batch_size, len(X_train), batch_size)):
            # get the inputs
            
            inputs = torch.from_numpy(X_train[start:end].as_matrix())
            inputs = inputs.float()
            labels = torch.Tensor(y_train[start:end])
            labels = labels.long()
    
            # wrap them in Variable
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()        
            optimizer.step()
            
            # print statistics
            running_loss += loss.data[0]

            if epoch % 100 == 99:
                print(loss.data[0])
                test_outputs = net(Variable(torch.from_numpy(X_test.as_matrix()).float()))
                _, predicted = torch.max(test_outputs.data, 1)
                accuracy = accuracy_score(y_test, predicted.numpy())
                print('%d loss: %.3f, accuracy: %.3f' % (epoch+1, running_loss / 100,
                      accuracy))
                running_loss = 0.0

    print('Finished Training')

In [62]:
torch.manual_seed(15)

net0 = SingleLayerNet()
optimizer0 = optim.SGD(net0.parameters(), lr=learning_rate, momentum=0.9)
torch.manual_seed(1234)

net1 = DoubleLayerNet()
optimizer1 = optim.SGD(net1.parameters(), lr=learning_rate, momentum=0.9)

train_with_net(net0, optimizer0, criterion, batch_size, epochs)

0.7567446827888489
100 loss: 0.008, accuracy: 0.347
0.7302234768867493
100 loss: 0.007, accuracy: 0.347
0.7375900745391846
100 loss: 0.007, accuracy: 0.347
0.7461714744567871
100 loss: 0.007, accuracy: 0.347
0.7207959890365601
100 loss: 0.007, accuracy: 0.347
0.7488070130348206
100 loss: 0.007, accuracy: 0.347
0.7610923647880554
100 loss: 0.008, accuracy: 0.347
0.7600045800209045
100 loss: 0.008, accuracy: 0.347
0.7349680066108704
100 loss: 0.007, accuracy: 0.347
0.745857298374176
100 loss: 0.007, accuracy: 0.347
0.7462821006774902
100 loss: 0.007, accuracy: 0.347
0.7566758990287781
100 loss: 0.008, accuracy: 0.347
0.7567446827888489
200 loss: 0.008, accuracy: 0.347
0.7302234768867493
200 loss: 0.007, accuracy: 0.347
0.7375900745391846
200 loss: 0.007, accuracy: 0.347
0.7461714744567871
200 loss: 0.007, accuracy: 0.347
0.7207959890365601
200 loss: 0.007, accuracy: 0.347
0.7488070130348206
200 loss: 0.007, accuracy: 0.347
0.7610923647880554
200 loss: 0.008, accuracy: 0.347
0.76000458002

In [59]:
train_with_net(net1, optimizer1, criterion, batch_size, epochs)

0.6196161508560181
100 loss: 0.006, accuracy: 0.738
0.5774780511856079
100 loss: 0.006, accuracy: 0.741
0.5789795517921448
100 loss: 0.006, accuracy: 0.741
0.532743513584137
100 loss: 0.005, accuracy: 0.741
0.609984815120697
100 loss: 0.006, accuracy: 0.741
0.6174442768096924
100 loss: 0.006, accuracy: 0.741
0.5354151129722595
100 loss: 0.005, accuracy: 0.741
0.5297208428382874
100 loss: 0.005, accuracy: 0.741
0.5572843551635742
100 loss: 0.006, accuracy: 0.741
0.5744524002075195
100 loss: 0.006, accuracy: 0.741
0.46535712480545044
100 loss: 0.005, accuracy: 0.741
0.5067234635353088
100 loss: 0.005, accuracy: 0.741
0.6165396571159363
200 loss: 0.006, accuracy: 0.741
0.5772078037261963
200 loss: 0.006, accuracy: 0.741
0.581137478351593
200 loss: 0.006, accuracy: 0.741
0.528733491897583
200 loss: 0.005, accuracy: 0.741
0.6111298203468323
200 loss: 0.006, accuracy: 0.741
0.6165362000465393
200 loss: 0.006, accuracy: 0.741
0.53464275598526
200 loss: 0.005, accuracy: 0.741
0.531524658203125

In [60]:
from sklearn.metrics import confusion_matrix
outputs = net0(Variable(torch.from_numpy(X_test.as_matrix()).float()))
_, predicted = torch.max(outputs.data, 1)
confusion = confusion_matrix(y_test, predicted.numpy())
accuracy = accuracy_score(y_test, predicted.numpy())


In [ ]:
confusion

In [ ]:
accuracy

In [50]:
class FourLayerNet(nn.Module):
    def __init__(self):
        super(FourLayerNet, self).__init__()
        self.fc1 = nn.Linear(7, 7)
        self.fc2 = nn.Linear(7, 2)
        self.fc3 = nn.Linear(7, 2)
        self.fc4 = nn.Linear(7, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = F.softmax(self.fc4(x))
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net2 = DoubleLayerNet()
optimizer2 = optim.SGD(net2.parameters(), lr=learning_rate, momentum=0.0)
train_with_net(net2, optimizer2, criterion, batch_size, epochs, epochs_to_print)

0.6304273009300232
10 loss: 0.063, accuracy: 0.738
0.5905195474624634
10 loss: 0.059, accuracy: 0.734
0.6098040342330933
10 loss: 0.061, accuracy: 0.734
0.5754178166389465
10 loss: 0.058, accuracy: 0.731
0.6121941208839417
10 loss: 0.061, accuracy: 0.731
0.5987313985824585
10 loss: 0.060, accuracy: 0.731
0.5567293763160706
10 loss: 0.056, accuracy: 0.731
0.5624191164970398
10 loss: 0.056, accuracy: 0.734
0.5905687212944031
10 loss: 0.059, accuracy: 0.734
0.5943408608436584
10 loss: 0.059, accuracy: 0.734
0.5206661820411682
10 loss: 0.052, accuracy: 0.734
0.5427858233451843
10 loss: 0.054, accuracy: 0.734
0.6213106513023376
20 loss: 0.062, accuracy: 0.744
0.5795345902442932
20 loss: 0.058, accuracy: 0.744
0.5891777873039246
20 loss: 0.059, accuracy: 0.744
0.5489501357078552
20 loss: 0.055, accuracy: 0.744
0.6055160760879517
20 loss: 0.061, accuracy: 0.741
0.6046742796897888
20 loss: 0.060, accuracy: 0.744
0.5444614291191101
20 loss: 0.054, accuracy: 0.744
0.5403470396995544
20 loss: 0.0

**MNIST**

In [92]:
from sklearn.datasets import fetch_mldata
mnist1d = fetch_mldata('MNIST original', data_home='./Data')

In [93]:
mnist1d.data.shape

(70000, 784)

In [94]:
mnist2d = np.reshape(mnist1d.data, (70000, 28, 28))

In [99]:
mnist2d.data.shape

(70000, 28, 28)

In [95]:
class MNISTImageNet(nn.Module):
    def __init__(self):
        super(MNISTImageNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

mnistnet = MNISTImageNet()
mnistoptimizer = optim.SGD(mnistnet.parameters(), lr=learning_rate, momentum=0.9)

In [98]:
def train_with_net(net, optimizer, criterion, batch_size, 
                   epochs):
    for epoch in range(epochs): # loop over the dataset multiple times
        
        running_loss = 0.0
        for start, end in zip(range(0, len(mnist2d.data), batch_size), 
                              range(batch_size, len(mnist2d.data), batch_size)):
            # get the inputs
            
            inputs = torch.from_numpy(mnist2d[start:end])
            inputs = inputs.float()
            labels = torch.from_numpy(mnist1d.target[start:end])
            labels = labels.long()
    
            # wrap them in Variable
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()        
            optimizer.step()
            
            # print statistics
            running_loss += loss.data[0]

            if epoch % 100 == 99:
                print(loss.data[0])
                test_outputs = net(Variable(torch.from_numpy(X_test.as_matrix()).float()))
                _, predicted = torch.max(test_outputs.data, 1)
                accuracy = accuracy_score(y_test, predicted.numpy())
                print('%d loss: %.3f, accuracy: %.3f' % (epoch+1, running_loss / 100,
                      accuracy))
                running_loss = 0.0

    print('Finished Training')
    
train_with_net(mnistnet, mnistoptimizer, criterion, batch_size, epochs)

AssertionError: 